In [1]:
from termcolor import colored
import ipywidgets as widgets
from datetime import date, datetime, timedelta
from traitlets import Unicode, validate

%run COVID_Info_functions.ipynb
%run Demographics_functions.ipynb

In [2]:
def Get_Data(State_Name, ByPass_Update):
    State, State_Code = Get_State_Code(State_Name)
    
    print(colored('Obtaining Counties Geographical Data for the State of ' + State_Name, 'red', attrs=['bold']))
    Counties_Geo_df = Counties_Info(State_Code)
    print(colored('Obtaining Congressional Districts Geographical Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_Geo_df = Congressional_Districts_Info(State_Code)
    
    print(colored('Obtaining Counties Demographics Data for the State of ' + State_Name, 'red', attrs=['bold']))
    Counties_Demo_df = Get_Counties_Demographics(State_Code)
    print(colored('Obtaining Congressional Districts Demographics Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_Demo_df = Get_Districts_Demographics(State, State_Code)

    print(colored('Obtaining Counties COVID Data for the State of ' + State_Name, 'red', attrs=['bold']))
    if State == 'VA':
        Counties_COVID_df = Get_Cleaned_VA_COVID_data(ByPass_Update)
    else:
        print(colored('COVID data is only available for the state of Virginia right now', 
                      'blue', 'on_yellow', attrs=['bold']))
        return State_Code, [Counties_Geo_df, Districts_Geo_df, Counties_Demo_df, Districts_Demo_df]
    
    print(colored('Computing Congressional Districts COVID Data for the State of ' + State_Name, 
                  'red', attrs=['bold']))
    Districts_COVID_df = Get_CongressionalDistricts_COVID_data(State_Code, Counties_COVID_df)
    report_date_list = Districts_COVID_df['Report Date'].unique()
    
    Counties_COVID_df = Add_Pop_data(State_Code, 'County', Counties_COVID_df, 
                                     Counties_COVID_df.loc[:, 'Total Cases':'New Monthly Deaths'].columns.to_list())
    Districts_COVID_df = Add_Pop_data(State_Code, 'Congressional District', Districts_COVID_df, 
                                      Districts_COVID_df.loc[:, 'Total Cases':'New Monthly Deaths'].columns.to_list())
    
    list_of_dfs = [Counties_Geo_df, Districts_Geo_df, Counties_Demo_df, Districts_Demo_df, 
                   Counties_COVID_df, Districts_COVID_df, report_date_list]
    
    return State_Code, list_of_dfs

In [3]:
def Get_Data_Map(State_Code, list_of_dfs):
    print(colored('Select options and click the Run button', 'red', 'on_yellow', attrs=['bold']))
    
    List_Keys_Dict = {('County', 'GEO'): 0, ('Congressional District', 'GEO'): 1, 
                      ('County', 'Partisan Voting Index'): 2, ('Congressional District', 'Partisan Voting Index'): 3,
                      ('County', 'Demographics, Age'): 2, ('Congressional District', 'Demographics, Age'): 3, 
                      ('County', 'Demographics, Race'): 2, ('Congressional District', 'Demographics, Race'): 3, 
                      ('County', 'Demographics, Income'): 2, ('Congressional District', 'Demographics, Income'): 3, 
                      ('County', 'COVID'): 4, ('Congressional District', 'COVID'): 5, 
                      ('None', 'None'): 7, ('County', 'None'): 7, ('Congressional District', 'None'): 7}
    
    if len(list_of_dfs) == 7:
        report_date_list = list_of_dfs[6]
    else:
        report_date_list = ['None']
        
    data_options = {'GEO': ['Population', 'Population Density', 'Land Area', 'Water Area'],
                    'COVID': ['Total Cases', 'Hospitalizations', 'Deaths', 
                              'New Daily Cases', 'New Daily Hospitalizations', 'New Daily Deaths', 
                              'Total Cases per Pop', 'Hospitalizations per Pop', 'Deaths per Pop', 
                              'New Daily Cases per Pop', 'New Daily Hospitalizations per Pop', 'New Daily Deaths per Pop', 
                              'Hospitalizations Ratio', 'Deaths Ratio', 'Deaths per Hospitalizations'], 
                    'Partisan Voting Index': ['PVI_R', 'PVI_D'], 
                    'Demographics, Age': ['Total Population', 'Under 5 years', '5 to 14 years', '15 to 24 years', 
                                          'Under 18 years', '25 to 34 years', 'Under 35 years', '35 to 44 years', 
                                          '25 to 44 years', 'Under 45 years', '45 to 54 years', '55 to 64 years', 
                                          '45 to 64 years', '65 to 74 years',  '75 to 84 years', '65 years and over', 
                                          '85 years and over', 
                                          'Percent Under 5 years', 'Percent 5 to 14 years', 'Percent 15 to 24 years', 
                                          'Percent Under 18 years', 'Percent 25 to 34 years', 'Percent Under 35 years', 
                                          'Percent 35 to 44 years', 'Percent 25 to 44 years', 'Percent Under 45 years', 
                                          'Percent 45 to 54 years', 'Percent 55 to 64 years', 'Percent 45 to 64 years', 
                                          'Percent 65 to 74 years',  'Percent 75 to 84 years', 'Percent 65 years and over', 
                                          'Percent 85 years and over'], 
                    'Demographics, Race': ['White, Not Hispanic', 'Asian, Not Hispanic', 
                                           'Black or African American, Not Hispanic', 
                                           'American Indian & Alaska Native, Not Hispanic',
                                           'Native Hawaiin & Other Pacific Islander, Not Hispanic', 
                                           'Hispanic or Latino', 'Other, Not Hispanic', 'Two or More Races, Not Hispanic', 
                                           'Percent White, Not Hispanic', 'Percent Asian, Not Hispanic', 
                                           'Percent Black or African American, Not Hispanic', 
                                           'Percent American Indian & Alaska Native, Not Hispanic', 
                                           'Percent Native Hawaiin & Other Pacific Islander, Not Hispanic', 
                                           'Percent Hispanic or Latino', 'Percent Other, Not Hispanic', 
                                           'Percent Two or More Races, Not Hispanic'], 
                    'Demographics, Income': ['Median Income', 'Median Income, Male', 'Median Income, Female', 
                                             'Per Capita Income', 'Median Household Income', 'Median Income Female/Male']}
    
    date_options = {'GEO':['CD116'], 
                    'COVID':report_date_list[::-1], 
                    'Partisan Voting Index': ['2008, 2012 elections'],
                    'Demographics, Age':['2019'], 
                    'Demographics, Race':['2019'], 
                    'Demographics, Income':['2019']}

    Log_Scale_W = widgets.Checkbox(value = False, description = 'Log Scale', indent = False)

    level_W = widgets.RadioButtons(options = ['County', 'Congressional District'], 
                                   description = 'Scope', 
                                   disabled = False)

    data_type_W = widgets.RadioButtons(options = data_options.keys(), 
                                       description = 'Data Type',  
                                       disabled=False)
    
    data_W = widgets.Dropdown(options = data_options[data_type_W.value])
    date_W = widgets.Dropdown(options = date_options[data_type_W.value])
    
    def update_data(*args):
        data_W.options = data_options[data_type_W.value]
        
    def update_date(*args):
        date_W.options = date_options[data_type_W.value]
        
    data_type_W.observe(update_data, 'value')
    data_type_W.observe(update_date, 'value')
    
    ##############################################################################################################
    def Map_State_Values(level, data_type, Log_Scale, date, data):
        k = List_Keys_Dict[(level, data_type)]
        
        if k >= len(list_of_dfs):
            return
        elif (k == 2) and (data_type == 'Partisan Voting Index'):
            print(colored('PVIs data for counties are not available', 'grey', 'on_yellow', attrs=['bold']))
            return
        
        if Log_Scale and (data_type == 'Partisan Voting Index'):
            Log_Scale = False
            print(colored('Log_Scale reset to False. PVI data have negative values!', 'grey', 'on_yellow', attrs=['bold']))
        
        df = list_of_dfs[k].copy()
        
        if k == 4 or k == 5:
            mask = df['Report Date'] == date
            df = df[mask]
            if k == 4:
                df.set_index('County Code', inplace = True)
            elif k == 5:
                df.set_index('CDistrict', inplace = True)
        
        return Map_State(State_Code, level, df, data, Log_Scale)
    ###############################################################################################################
    
    widgets.AppLayout(left_sidebar = level_W, right_sidebar = data_type_W, footer = data_W)
    widget = widgets.interactive(Map_State_Values, {'manual' : True, 'manual_name' : 'Run'}, 
                                 level = level_W, data_type = data_type_W, Log_Scale = Log_Scale_W, 
                                 date = date_W, data = data_W)
    
    controls = widgets.HBox(widget.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
    output = widget.children[-1]
    display(widgets.VBox([controls, output]))

In [4]:
def Get_Animated_COVID_Map(State_Code, list_of_dfs):
#     print(colored('Select options and click the Run button', 'red', 'on_yellow', attrs=['bold']))
    
    report_date_list = list_of_dfs[2]
    fmt = '%B %d %Y'
    
    report_dates = [(datetime.strptime(date, '%y/%m/%d').date()) for date in report_date_list]
    date_options = [(date.strftime(fmt), date) for date in report_dates]
    
    Log_Scale_W = widgets.Checkbox(value = False, description = 'Log Scale', indent = False)
    
    PerPop_W = widgets.Checkbox(value = False, description = 'Per Population', indent = False)

    scope_W = widgets.RadioButtons(options = ['Congressional District', 'County'],
                                   description = 'Scope',
                                   disabled = False)
    
    COVID_Data_W = widgets.RadioButtons(options = ['Cases', 'Hospitalizations', 'Deaths'], 
                                        description = 'Measure',
                                        disabled = False)
    
    Data_Count_W = widgets.RadioButtons(options = ['Total', 'Average'], 
                                        description = 'Count', 
                                        disabled = False)
    
    Dates_W = widgets.SelectionRangeSlider(options = date_options, 
                                           index = (0, len(date_options) - 1),
                                           layout = {'width': '700px', 'height': '80px'}, 
                                           style = {'description_width': '0px'},
                                           description = '',
                                           readout = False,
                                           disabled = False)
    
    Animated_Map_W = widgets.Checkbox(value = False, 
                                      description = 'Animated Map', 
                                      indent = False, 
                                      layout = {'width': '200px'})
    
    Plots_W = widgets.Checkbox(value = False, 
                               description = 'Time Series Plot', 
                               indent = False, 
                               layout = {'width': '200px'})
    
    Plot_Type_W = widgets.RadioButtons(options = ['scatter', 'line', 'bar'], 
                                       description = 'Plot Type', 
                                       disabled = False, 
                                       layout = {'visibility': 'hidden'})
    
    Normalized_W = widgets.Checkbox(value = False, 
                                    description = 'Normalized', 
                                    indent = False, 
                                    layout = {'visibility': 'hidden'})
    
    Add_State_W = widgets.Checkbox(value = False, 
                                   description = 'Compare to State-wide Values', 
                                   indent = False, 
                                   layout = {'visibility': 'hidden'})

    Wlist1 = ['From {}'.format((Dates_W.value[0]).strftime(fmt))]
    Wlist2 = ['From {}'.format(report_dates[0].strftime(fmt)), 
              '1 day', '7 days', '14 days', '21 days', '28 days']
    
    win_days_Dict = {'7 days': '7-day', '14 days': '14-day', '21 days': '21-day', '28 days': '28-day'}
    
    Window_W = widgets.Dropdown(options = Wlist1 + Wlist2, 
                                description = 'Window Size', 
                                layout = {'width': 'max-content'})
    
    SText_W = widgets.Label(value = 'Start Date:', layout = {'height': '20px'})
    EText_W = widgets.Label(value = 'End Date:', layout = {'height': '20px'})
    SLabel_W = widgets.Label(value = '{}'.format((Dates_W.value[0]).strftime(fmt)))
    ELabel_W = widgets.Label(value = '{}'.format((Dates_W.value[-1]).strftime(fmt)))
    
    freq_max = (Dates_W.value[-1] - Dates_W.value[0]).days
    freq_min = max(1, round(freq_max/120))

    Freq_W = widgets.BoundedIntText(
        value = freq_min,
        min = freq_min,
        max = freq_max,
        step = 1,
        description = 'Animation frequency in number of days', 
        style = {'description_width': '230px'}, 
        layout = {'height': '40px', 'visibility': 'hidden'},
#         layout={'width': '330px', 'height': '40px'},
        disabled = False
    )
    
    Nframes = 1 + round(((Dates_W.value[-1] - Dates_W.value[0]).days)/Freq_W.value)
    FLabel_W = widgets.Label(value = 'Number of Animation frames = {}'.format(Nframes), 
                             layout = {'visibility': 'hidden'})
    
    def update_Slabel(*args):
        SLabel_W.value = '{}'.format((Dates_W.value[0]).strftime(fmt))
        
    def update_Elabel(*args):
        ELabel_W.value = '{}'.format((Dates_W.value[-1]).strftime(fmt))
        
    def update_Window(*args):
        Wlist1 = ['From {}'.format((Dates_W.value[0]).strftime(fmt))]
        Window_W.options = Wlist1 + Wlist2
        
    def update_Freq(*args):
        if Dates_W.value[-1] <= Dates_W.value[0]:
            freq_max = 0
            freq_min = 0
        else:
            freq_max = (Dates_W.value[-1] - Dates_W.value[0]).days
            freq_min = max(1, round(freq_max/120))
            
        Freq_W.value = freq_min
        if freq_min > Freq_W.max:
            Freq_W.max = freq_max
            Freq_W.min = freq_min
        else:
            Freq_W.min = freq_min
            Freq_W.max = freq_max
            
        if Animated_Map_W.value:
            Freq_W.layout.visibility = 'visible'
        else:
            Freq_W.layout.visibility = 'hidden'
        
    def update_Flabel(*args):
        if Dates_W.value[-1] <= Dates_W.value[0]:
            Nframes = 1
        else:
            Nframes = 1 + round(((Dates_W.value[-1] - Dates_W.value[0]).days)/Freq_W.value)
        FLabel_W.value = "Number of Animation frames = {}".format(Nframes)
        
        if Animated_Map_W.value:
            FLabel_W.layout.visibility = 'visible'
        else:
            FLabel_W.layout.visibility = 'hidden'
            
    def update_Plot_Type(*args):
        if Plots_W.value:
            Plot_Type_W.layout.visibility = 'visible'
        else:
            Plot_Type_W.layout.visibility = 'hidden'
            
    def update_Normalized(*args):
        if Plots_W.value:
            Normalized_W.layout.visibility = 'visible'
        else:
            Normalized_W.layout.visibility = 'hidden' 
            
    def update_Add_State(*args):
        if Plots_W.value:
            Add_State_W.layout.visibility = 'visible'
        else:
            Add_State_W.layout.visibility = 'hidden'    
        
    Dates_W.observe(update_Slabel, 'value')
    Dates_W.observe(update_Elabel, 'value')
    Dates_W.observe(update_Window, 'value')
    Dates_W.observe(update_Freq, 'value')
    Dates_W.observe(update_Flabel, 'value')
    
    Animated_Map_W.observe(update_Freq, 'value')
    Animated_Map_W.observe(update_Flabel, 'value')
    
    Freq_W.observe(update_Flabel, 'value')
    
    Plots_W.observe(update_Plot_Type, 'value')
    Plots_W.observe(update_Normalized, 'value')
    Plots_W.observe(update_Add_State, 'value')
    
    ##############################################################################################################
    def Animated_Map_Values(scope, COVID_Data, Data_Count, Window, Per_Pop, Log_Scale, Dates, SLabel, ELabel, 
                            SText, EText, freq, FLabel, Animated_Map, Plots, Plot_Type, Normalized, Add_State):
        sdate = Dates[0]
        edate = Dates[-1]
    
        if edate <= sdate:
            Nframes = 0
            delta_dates = 0
        else:
            delta_dates = (edate - sdate).days
            Nframes = round((delta_dates)/freq)
            ffreq = ((edate - sdate).days)/Nframes
       
        if Nframes > 0:
            rdates = [(sdate + timedelta(days = int(i*ffreq))).strftime('%y/%m/%d') for i in range(Nframes + 1)]
            wdates = [(sdate + timedelta(days = i)).strftime('%y/%m/%d') for i in range(delta_dates + 1)]
        else:
            rdates = [edate.strftime('%y/%m/%d')]
            wdates = [edate.strftime('%y/%m/%d')]
        
        if scope == 'County':
            df = list_of_dfs[0].copy()
            col_list = ['Report Date', 'County Name', 'Total '+ COVID_Data, 'New Daily ' + COVID_Data]
            sIndex = 'County Code'
        elif scope == 'Congressional District':
            df = list_of_dfs[1].copy()
            col_list = ['Report Date', 'Total '+ COVID_Data, 'New Daily ' + COVID_Data]
            sIndex = 'CDistrict'

        df.set_index(sIndex, inplace = True)
        df.rename(columns={"Hospitalizations": "Total Hospitalizations", "Deaths": "Total Deaths"}, inplace = True)
        
        df.drop(df.columns.difference(col_list), axis = 1, inplace = True)
        mask = df['Report Date'] == report_date_list[0]
        df.loc[mask, 'New Daily ' + COVID_Data] = df.loc[mask, 'Total ' + COVID_Data]
        
        df_state = df.groupby('Report Date').sum()
        
        if Window == '1 day':
            New_Col = 'Daily ' + COVID_Data
            df[New_Col] = df['New ' + New_Col]
            df_state[New_Col] = df_state['New ' + New_Col]
        else:
            if Data_Count == 'Total':
                New_Col = COVID_Data
            elif Data_Count == 'Average':
                New_Col = 'Average ' + COVID_Data
            else:
                print('Error: Count value should be either \"Total" or \"Average" ')
                return
        
        if Window in Wlist2[2:]:
            win = int(Window.split()[0])
            if Data_Count == 'Total':
                df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                               rolling(win, min_periods = 1).sum().reset_index(1, drop = True).astype(int))
                df_state[New_Col] = (df_state['New Daily ' + COVID_Data].
                                     rolling(win, min_periods = 1).sum().astype(int))
            elif Data_Count == 'Average':
                df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                               rolling(win, min_periods = 1).mean().reset_index(1, drop = True).round(decimals = 2))
                df_state[New_Col] = (df_state['New Daily ' + COVID_Data].
                                     rolling(win, min_periods = 1).mean().round(decimals = 2))
        elif Window == Wlist2[0]:
            if Data_Count == 'Total':
                df[New_Col] = df['Total ' + COVID_Data]
                df_state[New_Col] = df_state['Total ' + COVID_Data]
            elif Data_Count == 'Average':
                df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                               expanding().mean().reset_index(1, drop = True).round(decimals = 2))
                df_state[New_Col] = df_state['New Daily ' + COVID_Data].expanding().mean().round(decimals = 2)
        
        df_all = df[df['Report Date'].isin(wdates)]
        df_state = df_state[df_state.index.get_level_values(0).isin(wdates)]
        
        df = df[df['Report Date'].isin(rdates)]
        
        if New_Col not in df:
            if Window == 'From {}'.format(sdate.strftime(fmt)):
                if Data_Count == 'Total':
                    df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                   expanding().sum().reset_index(1, drop = True).astype(int))
                    df_all[New_Col] = (df_all.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                       expanding().sum().reset_index(1, drop = True).astype(int))
                    df_state[New_Col] = (df_state['New Daily ' + COVID_Data].expanding().sum().astype(int))
                elif Data_Count == 'Average':
                    df[New_Col] = (df.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                   expanding().mean().reset_index(1, drop = True).round(decimals = 2))
                    df_all[New_Col] = (df_all.reset_index().groupby(sIndex)['New Daily ' + COVID_Data].
                                       expanding().mean().reset_index(1, drop = True).round(decimals = 2))
                    df_state[New_Col] = (df_state['New Daily ' + COVID_Data].expanding().mean().round(decimals = 2))                    
            else:
                print('Error in code: Where is the ', New_Col, ' column?!')
                return
            
        if Window.startswith('From'):
            win_date = Window.split(' ', 1)[1]
            title1 = 'Running ' + Data_Count
            title3 = ' since ' + win_date
        elif Window == '1 day':
            title1 = 'New Daily'
            title3 = ''
        else:
            if Data_Count == 'Total':
                title1 = 'Total Number of'
                title3 = ' in the last ' + Window
            elif Data_Count == 'Average':
                title1 = win_days_Dict[Window] + ' Rolling Average'
                title3 = ''
            
        if Per_Pop:
            data_col = New_Col + '/10000 pop'
            df[data_col] = (Get_Per_Pop(State_Code, scope, df, New_Col))*10000
            df_all[data_col] = (Get_Per_Pop(State_Code, scope, df_all, New_Col))*10000
            State_Pop = Get_State_Pop19(State_Code)
            df_state[data_col] = (df_state[New_Col]/State_Pop.loc[State_Code, 'Population'])*10000
            title2 = ' ' + COVID_Data + ' per 10,000 pop'
        else:
            data_col = New_Col
            title2 = ' ' + COVID_Data
            
        title = title1 + title2 + title3
        
        if Animated_Map:
            Map_Fig = Animated_Map_State(State_Code, scope, df, data_col, 'Report Date', title, Log_Scale)
        
        if Plots:
            if Normalized:
                df_all['Normalized ' + data_col] = (df_all.loc[:, data_col]/df_all.reset_index().
                                  groupby(sIndex)[data_col].max()).replace(np.inf, 0)
                df_state['Normalized ' + data_col] = (df_state.loc[:, data_col]/df_state[data_col].max()
                                                     ).replace(np.inf, 0)
                data_col = 'Normalized ' + data_col
                title = 'Normalized ' + title
                
            if Add_State:
                df_all.reset_index(inplace = True)
                df_all.set_index('Report Date', inplace = True)
                N_locations = len(df_all[sIndex].unique())
                df_all = pd.concat([df_state, df_all]).fillna('State-Wide')
                df_all.sort_index(inplace = True)
            
                if Normalized or data_col[-3:] == 'pop':
                    df_all[data_col + ' minus State-Wide value'] = (df_all.loc[:, data_col] - df_state.loc[:, data_col])
                    
                    df_all.reset_index(inplace = True)
                    df_all.sort_values(by = [sIndex, 'Report Date'], inplace = True)
                    df_all.set_index(sIndex, inplace = True)
                    
                    Plot_Fig = TimeSeries_Norm_Plots(State_Code, scope, Plot_Type, df_all, data_col, Log_Scale, title)
                else:
                    df_state_Av = df_state/N_locations
                    df_all = pd.concat([df_state_Av, df_all]).fillna('State Average')
                    df_all.sort_index(inplace = True)
                    df_all['Diff'] = (df_all.loc[:, data_col] - df_state_Av.loc[:, data_col])
                    
                    df_all['Ratio'] = (df_all.loc[:, 'Diff']/df_state.loc[:, data_col]).replace(np.inf, 0)
                    
                    df_all.reset_index(inplace = True)
                    df_all.sort_values(by = [sIndex, 'Report Date'], inplace = True)
                    df_all.set_index(sIndex, inplace = True)
                    
                    df_all.loc['State-Wide', 'Diff'] = np.nan
                    df_all.loc['State-Wide', 'Ratio'] = np.nan
                    
                    Plot_Fig = TimeSeries_Multi_Plots(State_Code, scope, Plot_Type, df_all, data_col, Log_Scale, title)
            else:
                Plot_Fig = TimeSeries_Plot(State_Code, scope, Plot_Type, df_all, data_col, Log_Scale, title)
                
        return
    ###############################################################################################################
    
    widget = widgets.interactive(Animated_Map_Values, {'manual' : True, 'manual_name' : 'Run'}, 
                                 scope = scope_W, COVID_Data = COVID_Data_W, Data_Count = Data_Count_W, 
                                 Window = Window_W, Per_Pop = PerPop_W, Log_Scale = Log_Scale_W, Dates = Dates_W, 
                                 SLabel = SLabel_W, ELabel = ELabel_W, SText = SText_W, EText = EText_W, 
                                 freq = Freq_W, FLabel = FLabel_W, Animated_Map = Animated_Map_W, Plots = Plots_W, 
                                 Plot_Type = Plot_Type_W, Normalized = Normalized_W, Add_State = Add_State_W)
    
    col1_1 = widgets.VBox([SText_W, SLabel_W])
    col1_2 = widgets.VBox([EText_W, ELabel_W])
    row1 = widgets.HBox(children=[col1_1, Dates_W, col1_2])

    col2 = widgets.VBox([PerPop_W, Log_Scale_W])
    row2 = widgets.HBox([scope_W, COVID_Data_W, col2, Data_Count_W, Window_W], 
                        layout = widgets.Layout(flex_flow = 'row wrap'))
    
    col3 = widgets.HBox([Freq_W, FLabel_W])
    row3 = widgets.HBox([Animated_Map_W, Freq_W, FLabel_W], 
                        layout = widgets.Layout(display = 'flex', 
                                                justify_content = 'space-between', 
                                                width = '90%'))
    
    col4 = widgets.VBox([Normalized_W, Add_State_W])
    
    row4 = widgets.HBox([Plots_W, Plot_Type_W, col4])
    
    output = widget.children[-1]
    display(widgets.VBox([row1, row2, row3, row4, widget.children[-2], output], 
                         layout = widgets.Layout(display="flex", justify_content="flex-start")))
    